# Desafío Complementario: CrossValidation y mejora de modelos de ML

In [58]:
# Importacion de librerias para manipulacion de datos
import pandas as pd
import numpy as np

In [59]:
# Importacion de Plotly como libreria de DataViz
import plotly.graph_objects as go

In [60]:
# Importacion de Sklearn para la costruccion de modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc

#### Configurando path

In [61]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [62]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\\data\procesed\blueTeam.csv")
redTeam = pd.read_csv( path_base + r"\\data\procesed\redTeam.csv")
data = pd.concat([blueTeam,redTeam])

In [63]:

# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}
data.drop(columns=["Unnamed: 0"], inplace=True)

### Preparar datos:


In [64]:

X = blueTeam.drop('Wins', axis=1)  
y = blueTeam['Wins']


In [65]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Crear modelo de Random Forest:

In [66]:
# Aplicar PCA
n_components = 10  # Número de componentes principales
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [67]:
# Entrenar un modelo de Random Forest en los datos transformados por PCA
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_pca, y_train)

RandomForestClassifier(random_state=42)

### Aplicar validación cruzada:

In [68]:
# Utiliza el número de divisiones y la métrica deseada
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')  

### Visualizar resultados:

In [69]:
print(f'Puntuaciones de Validación Cruzada: {cv_scores}')
print(f'Precisión Promedio: {cv_scores.mean()}')

Puntuaciones de Validación Cruzada: [0.69736842 0.71406883 0.71609312 0.72925101 0.71848101]
Precisión Promedio: 0.7150524778352894


In [70]:
# Visualizar resultados con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, 6)), y=cv_scores, mode='lines+markers', name='Puntuaciones de Validación Cruzada'))

fig.update_layout(
    title='Puntuaciones de Validación Cruzada para Random Forest',
    xaxis_title='Iteración de Validación Cruzada',
    yaxis_title='Precisión',
    showlegend=True,
)

fig.show()

### Aplicando Curva de ROC

In [71]:
# Obtener probabilidades de predicción para la clase positiva
y_probs = rf_model.predict_proba(X_test_pca)[:, 1]

In [72]:
# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

In [73]:
# Crear la figura interactiva de Plotly
fig = go.Figure()

# Agregar la curva ROC
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'Curva ROC (AUC = {roc_auc:.2f})'))

# Línea de referencia
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Línea de Referencia', line=dict(dash='dash')))

# Diseño del gráfico
fig.update_layout(
    title='Curva ROC con PCA y Random Forest',
    xaxis_title='Tasa de Falsos Positivos (FPR)',
    yaxis_title='Tasa de Verdaderos Positivos (TPR)',
    legend=dict(x=0.02, y=0.95),
    margin=dict(l=0, r=0, t=30, b=0)
)

# Mostrar la figura interactiva
fig.show()

In [74]:
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión interactiva con Plotly
fig = ff.create_annotated_heatmap(z=cm, x=['Predicción Negativa', 'Predicción Positiva'], y=['Real Negativa', 'Real Positiva'], colorscale='Viridis')

# Diseño del gráfico
fig.update_layout(
    title='Matriz de Confusión',
    xaxis=dict(title='Predicción'),
    yaxis=dict(title='Realidad'),
)

# Mostrar la figura interactiva
fig.show()